In [1]:
# importing libraries
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import ast

In [2]:
# load_dataset("lukebarousse/data_jobs")

In [ ]:
data = load_dataset("lukebarousse/data_jobs")

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

data_jobs.csv:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
df = data["train"].to_pandas()
# cleaning data
df["job_posted_date"] = pd.to_datetime(df["job_posted_date"])
df.loc[:,"job_skills"] = df.loc[:,"job_skills"].apply(func=(lambda skills_list : ast.literal_eval(skills_list) if pd.notna(skills_list) else skills_list))
df_exploded = df.explode(column="job_skills").copy()

In [ ]:
del df
df_exploded.info()

In [ ]:
df_skill_count = (df_exploded
                  .groupby(by=["job_title_short", "job_skills"])
                  .size()
                  .reset_index()
                  .rename(columns={0:"skill_count"})
                  .sort_values(by="skill_count", ascending=False))

In [ ]:
job_title = "Data Analyst"
top_skills_count = 10
role_top_skills = df_skill_count[df_skill_count["job_title_short"] == job_title].head(top_skills_count)
role_top_skills.plot(kind="barh", x="job_skills", y="skill_count")
plt.gca().invert_yaxis()
plt.title(f"Top {top_skills_count} for {job_title}s")
plt.xlabel("Job Posting Count")
plt.ylabel("")
plt.legend().set_visible(False)
plt.show()

# Excercise - Investigate Trending Skills

In [ ]:
df_DS = df_exploded[df_exploded["job_title_short"] == "Data Scientist"].copy()

In [ ]:
df_DS["job_posted_month_no"] = df_DS["job_posted_date"].dt.month

In [ ]:
df_DS_pv = df_DS.pivot_table(index="job_posted_month_no", columns="job_skills", aggfunc="size", fill_value=0)
df_DS_pv

In [ ]:
# df_DS_pv.sum()
df_DS_pv.loc["Total"] = df_DS_pv.sum()

In [ ]:
df_DS_pv = df_DS_pv.T.sort_values(by="Total", ascending=False).T
df_DS_pv.drop(index="Total", inplace=True)

In [ ]:
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
             7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df_DS_pv.index = df_DS_pv.index.map(month_map)

In [ ]:
df_DS_pv.iloc[:,:5].plot()
plt.title("Top 5 Trending Skill of Data Scientists")
plt.ylabel("Count")
plt.xlabel("")
plt.show()

# Ploting Top Job Postings

In [ ]:
top_posts_count = 3
top_job_postings = df_exploded["job_title_short"].value_counts().sort_values(ascending=False).head(top_posts_count).index.to_list()
fig, ax = plt.subplots(3,1, figsize=(14, 7))
for i,job_title in enumerate(top_job_postings):
    df_role = df_exploded[df_exploded["job_title_short"] == job_title]
    df_role_skill_count = df_role["job_skills"].value_counts()
    df_role_skill_count.head(5).plot(kind="barh", ax=ax[i], title=job_title)
    ax[i].set_ylabel("")
    ax[i].set_xlabel("Skill Count")
    ax[i].invert_yaxis()
    ax[i].set_xlim(0, 120000)
fig.suptitle("Top 3 Postings Skills", fontsize=15)
fig.tight_layout()


In [ ]:
import matplotlib.pyplot as plt

def plot_top_job_skills(df, top_n=3, xlim=(0, 120000)):
    # Get the top job postings
    top_job_titles = df["job_title_short"].value_counts().head(top_n).index.to_list()
    
    fig, ax = plt.subplots(top_n, 1, figsize=(14, 7))
    
    for i, job_title in enumerate(top_job_titles):
        df_filtered = df[df["job_title_short"] == job_title]
        skill_counts = df_filtered["job_skills"].value_counts().head(5)
        
        skill_counts.plot(kind="barh", ax=ax[i], title=job_title)
        ax[i].set_ylabel("")
        ax[i].set_xlabel("Skill Count")
        ax[i].invert_yaxis()
        ax[i].set_xlim(*xlim)  # Set x-axis limits dynamically
    
    fig.suptitle("Top Job Postings Skills", fontsize=15)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to make room for the suptitle
    plt.show()

# Usage
plot_top_job_skills(df_exploded)

# Excercise - Top Companies hires for roles of ?

In [ ]:
companies =  df_exploded["company_name"].value_counts().head(5).index.to_list()
companies

In [ ]:
df_companies_pvt = df_exploded.pivot_table(index="company_name", columns="job_title_short", aggfunc="size", fill_value=0).loc[companies]

In [ ]:
df_companies_pvt.loc["Total"] = df_companies_pvt.sum()

In [ ]:
df_companies_pvt_top_n = df_companies_pvt.T.sort_values(by="Total", ascending=False).T.drop(index="Total")

In [ ]:
df_top_n_company_roles = df_companies_pvt_top_n.iloc[:,:5]
df_top_n_company_roles.plot(kind="barh")
plt.show()

In [ ]:
df_top_n_company_roles.loc[companies[0]]

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(10,10))
for i,company in enumerate(companies):
    df_top_n_company_roles.loc[company].plot(kind="barh", ax=ax[i])
    ax[i].set_xlim(0,13000)
    ax[i].set_title(company.upper())
    ax[i].set_xlabel("Count")
    ax[i].set_ylabel("")
    
fig.suptitle("Companies Top Postings")
fig.tight_layout()
plt.show()

# Pie Chart

In [ ]:
(df_exploded["job_health_insurance"]
 .value_counts()
 .plot(
     kind="pie", 
     autopct = "%1.1f%%", 
     explode=[0, 0.1], 
     colors=['#ff9999','#66b3ff'], 
     title="Health Insurance Distribution",
     shadow=True,
     startangle=90
 ))
plt.show()

In [ ]:
columns_bool = df_exploded.select_dtypes(include=bool).columns.to_list()
columns_bool_dict = {col:" ".join([col_in.title() for col_in in col.split(sep="_")]) for col in columns_bool}

In [ ]:
fig, ax = plt.subplots(1, len(columns_bool)) 
for i,(column, title) in enumerate(columns_bool_dict.items()):
    (ax[i]
     .pie(df_exploded[column]
          .value_counts(), 
          autopct="%1.1f",
          startangle=90,
          explode=[0.1, 0],
          labels=["False", "True"]))
    ax[i].set_title(title)
fig.tight_layout()
plt.show()

# Median of Salary vs Count of Skills

In [ ]:
df_sal_median_count = df_exploded.pivot_table(index="job_skills", aggfunc={"salary_year_avg": ["median", "count"]})
df_sal_median_count.columns = ["Count", "Median"]

In [ ]:
df_sal_mc_10 = df_sal_median_count.sort_values(by="Count", ascending=False).head(10)
df_sal_mc_10.index[0]

In [ ]:
plot = plt.scatter(df_sal_mc_10["Count"], df_sal_mc_10["Median"])
for i,(x,y) in enumerate(zip(df_sal_mc_10["Count"], df_sal_mc_10["Median"])):
    plt.text(x,y,df_sal_mc_10.index[i])
plt.xlabel("Count")
plt.ylabel("Median Yearly Salary")
plt.show()

# Online Method

In [ ]:
df_DS = df_exploded[df_exploded["job_title_short"]=="Data Analyst"]
df_DS.head()

In [ ]:
df_DS_gb = df_DS.groupby("job_skills").agg(
    skill_count = ("job_skills", "count"),
    median_salary = ("salary_year_avg", "median")
)

In [ ]:
df_DS_gb_df = df_DS_gb.sort_values(by="skill_count", ascending=False)

In [ ]:
plot_df = df_DS_gb_df.head(10)
plot_df.plot(kind="scatter", x="skill_count", y="median_salary", )
for i,(x,y) in enumerate(zip(plot_df["skill_count"], plot_df["median_salary"])):
    plt.text(x,y,plot_df.index[i])
plt.xlabel("Count")
plt.ylabel("Median Yearly Salary")
plt.show()